In [ ]:
import pathlib
from sqlalchemy.orm import declarative_base, sessionmaker, registry
from sqlalchemy import Column, String, DateTime, Integer, PickleType, Table, ForeignKey, create_engine
from sqlalchemy.ext.mutable import MutableList
import datetime

In [ ]:
import pathlib
from sqlalchemy.orm import sessionmaker, registry
from sqlalchemy import create_engine
from contextlib import contextmanager


def create_database():
    """
    Creates databases for each class as a SQLite database using SQL Alchemy's ORM module.
    :return: None
    """
    BASE_DIR = pathlib.Path().absolute()

    # Create a connection string to the database
    connection_string = "sqlite:///" + pathlib.Path.joinpath(BASE_DIR, "routing.db").as_posix()

    # Create engine that will be used to file to the SQLite database. Echo allows us to bypass comments produced by SQL
    engine = create_engine(connection_string, echo=True)

    # Create and configure a sessionmaker class which we use to populate each individual table.
    Session = sessionmaker(bind=engine)

    # Create mapper registry
    mapper_registry = registry()

    return engine, Session, mapper_registry


def create_tables(engine, base):
    """
    Creates tables based on metadata provided. This will convert the __table__ attribute
    :param engine: Database engine for filing changes to DB
    :param base: Base to collect table metadata from
    :return: None
    """
    base.metadata.create_all(engine)



def write_obj(obj):
    """
    Creates a new row in a table. Corresponding table is specified in the class of the object passed to this function.
    :param obj: Object to write to table. The class of the object specifies the parameters of the table.
    :return: None
    """
    # Set context to use the session that is bound to our engine for this transaction
    with session_scope() as local_session:
        # Add the row to the table
        local_session.merge(obj)


def load_obj(cls, obj_id):
    """
    loads a new row in a table. Corresponding table is specified in the class of the object passed to this function.
    :param obj: Object to write to table. The class of the object specifies the parameters of the table.
    :return: None
    """
    # Set context to use the session that is bound to our engine for this transaction
    with session_scope() as local_session:
        # Add the row to the table
        obj = local_session.query(cls).filter(cls._id == obj_id).first()

    return obj


@contextmanager
def session_scope():
    """
    Provides a transactional scope around a series of operations.
    :return: None
    """
    session = Session()
    try:
        yield session
        session.commit()
    except:
        session.rollback()
        raise
    finally:
        session.close()


# initialize SQLAlchemy objects
engine, Session, mapper_registry = create_database()

In [ ]:
@mapper_registry.mapped
class Visit:
    __table__ = Table(
        "Visit",
        mapper_registry.metadata,
        Column("_id", Integer, primary_key=True, unique=True),
        Column("pat_id", Integer),
        Column("clin_id", Integer, nullable=True),
        Column("exp_date", DateTime, nullable=True),
        Column("status", String, nullable=True),
        Column("time_earliest", DateTime, nullable=True),
        Column("time_latest", DateTime, nullable=True),
        Column("visit_priority", String, nullable=True),
        Column("visit_complexity", String, nullable=True),
        Column("skill_list", MutableList.as_mutable(PickleType), nullable=True),
        Column("discipline", String, nullable=True),
        Column("cancel_reason", String, nullable=True)
    )

    _c_visit_complexity = ("simple", "routine", "complex")
    _c_visit_priority = ("green", "amber", "red")
    _c_sched_status = ("unassigned", "assigned", "no show", "cancelled")
    _c_cancel_reason = ("clinician unavailable", "patient unavailable", "no longer needed", "expired", "system action")

    def __init__(self, pat_id=10000, clin_id=None, status=1, sched_status="unscheduled", time_earliest="2000", time_latest="2200",
                 exp_date="22/12/2020", visit_complexity=_c_visit_complexity[1], visit_priority=_c_visit_priority[0],
                 skill_list=[1, 2, 3], discipline="", **kwargs):
        """Initializes a new request and links with pat_id. It contains the following attributes:
            req_id, pat_id, name, status, the earliest time, latest time, sched status, and cancel_reason"""
        self._id = 10000
        self.exp_date = datetime.datetime.strptime(exp_date, "%d/%m/%Y")
        self.pat_id = pat_id
        self.clin_id = clin_id
        self._name = "Visit" + str(self._id)
        self.status = status
        self.time_earliest = datetime.datetime.strptime(time_earliest, "%H%M")
        self.time_latest = datetime.datetime.strptime(time_latest, "%H%M")
        self.visit_priority = visit_priority
        self.visit_complexity = visit_complexity
        self.skill_list = skill_list
        self.discipline = discipline
        self._cancel_reason = None
        self._sched_status = sched_status

In [ ]:
create_tables(engine, mapper_registry)

In [ ]:
write_obj(Visit())

In [ ]:
obj = load_obj(Visit, 10000)

In [ ]:
with session_scope() as local_session:
    # Add the row to the table
    obj = local_session.query(Visit).filter(Visit._id == 10000).first()
    obj.discipline = "Nurse"

In [7]:
from classes.person import Patient, Clinician
from classes.visits import Visit
from classes.team import Team

In [ ]:
from data_manager import DataManagerMixin

DataManagerMixin.create_tables()


In [3]:
Patient.load_tracked_instances()
Team.load_tracked_instances()
Visit.load_tracked_instances()
Clinician.load_tracked_instances()

In [4]:

visit = Visit(10000, exp_date="22/12/2022", time_earliest="0800", time_latest="1700")


Patient could not be found.

Clinician could not be found.
Patient successfully saved.



In [5]:
visit.write_obj()

2022-12-21 14:30:52,867 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-21 14:30:52,871 INFO sqlalchemy.engine.Engine SELECT "Visit"._id AS "Visit__id", "Visit"._pat_id AS "Visit__pat_id", "Visit"._clin_id AS "Visit__clin_id", "Visit"._exp_date AS "Visit__exp_date", "Visit"._status AS "Visit__status", "Visit"._time_earliest AS "Visit__time_earliest", "Visit"._time_latest AS "Visit__time_latest", "Visit"._visit_priority AS "Visit__visit_priority", "Visit"._visit_complexity AS "Visit__visit_complexity", "Visit"._skill_list AS "Visit__skill_list", "Visit"._discipline AS "Visit__discipline", "Visit"._cancel_reason AS "Visit__cancel_reason", "Visit"._sched_status AS "Visit__sched_status" 
FROM "Visit" 
WHERE "Visit"._id = ?
2022-12-21 14:30:52,872 INFO sqlalchemy.engine.Engine [generated in 0.00109s] (10001,)
2022-12-21 14:30:52,875 INFO sqlalchemy.engine.Engine UPDATE "Visit" SET _exp_date=?, _status=?, _time_earliest=?, _time_latest=?, _visit_priority=?, _visit_complexity=?, _disci

In [9]:
new_visit = Visit.load_obj(10001)

2022-12-21 16:00:05,173 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-21 16:00:05,177 INFO sqlalchemy.engine.Engine SELECT "Visit"._id AS "Visit__id", "Visit"._pat_id AS "Visit__pat_id", "Visit"._clin_id AS "Visit__clin_id", "Visit"._exp_date AS "Visit__exp_date", "Visit"._status AS "Visit__status", "Visit"._time_earliest AS "Visit__time_earliest", "Visit"._time_latest AS "Visit__time_latest", "Visit"._visit_priority AS "Visit__visit_priority", "Visit"._visit_complexity AS "Visit__visit_complexity", "Visit"._skill_list AS "Visit__skill_list", "Visit"._discipline AS "Visit__discipline", "Visit"._cancel_reason AS "Visit__cancel_reason", "Visit"._sched_status AS "Visit__sched_status" 
FROM "Visit" 
WHERE "Visit"._id = ?
 LIMIT ? OFFSET ?
2022-12-21 16:00:05,178 INFO sqlalchemy.engine.Engine [generated in 0.00108s] (10001, 1, 0)


In [ ]:
new_visit.discipline

In [ ]:
new_visit.discipline="Physical Therapist"
new_visit.discipline

In [ ]:
new_visit.session.rollback()
new_visit.session.refresh(new_visit)


In [ ]:
new_visit.discipline

In [37]:
new_visit.session.query(Visit).filter(Visit._id==10030).all()

2022-12-21 16:50:44,341 INFO sqlalchemy.engine.Engine SELECT "Visit"._id AS "Visit__id", "Visit"._pat_id AS "Visit__pat_id", "Visit"._clin_id AS "Visit__clin_id", "Visit"._exp_date AS "Visit__exp_date", "Visit"._status AS "Visit__status", "Visit"._time_earliest AS "Visit__time_earliest", "Visit"._time_latest AS "Visit__time_latest", "Visit"._visit_priority AS "Visit__visit_priority", "Visit"._visit_complexity AS "Visit__visit_complexity", "Visit"._skill_list AS "Visit__skill_list", "Visit"._discipline AS "Visit__discipline", "Visit"._cancel_reason AS "Visit__cancel_reason", "Visit"._sched_status AS "Visit__sched_status" 
FROM "Visit" 
WHERE "Visit"._id = ?
2022-12-21 16:50:44,344 INFO sqlalchemy.engine.Engine [cached since 61.73s ago] (10030,)


[]

In [ ]:
Team.import_csv("./investigation/team_import.csv")

In [ ]:
Patient.import_csv("./investigation/pat_import.csv")

In [ ]:
Visit.import_csv("./investigation/visit_import.csv")

In [ ]:
Clinician.import_csv("./investigation/clin_import.csv")